In [1]:
from sqlalchemy import create_engine
import sqlalchemy
import pandas as pd
import os
from datetime import datetime, timedelta
from dotenv import load_dotenv


In [2]:
from oceanoobsbrasil.bd import Getdata
%load_ext autoreload
%autoreload 2

In [3]:
bd = Getdata()

In [4]:
bd.get(table='stations', institution=['=', 'simcosta'], data_type=['=', 'buoy'])

,id,name,lat,lon,data_type,institution,url
0,897,RJ3,-22.9831,-43.1745,buoy,simcosta,3
1,898,RS2,-32.1346,-52098.0000,buoy,simcosta,6
2,899,RS5,-32296.0000,-52.0239,buoy,simcosta,10
3,900,RJ4,-22.9715,-43.1522,buoy,simcosta,12
4,901,RS3,-32195.0000,-52.0794,buoy,simcosta,13
5,902,RS4,-32.2454,-52.0954,buoy,simcosta,14
6,903,BA1,-12.9895,-37.4582,buoy,simcosta,15


In [8]:
df = bd.get(table='data_stations', station_id=['in', [902]])

In [9]:
df.shape

(94, 22)

In [7]:
bd.post(table='data_stations', df=df)

DELETE FROM data_stations WHERE true AND station_id in (902) AND date_time >= '2021-09-06 00:21:00'


In [47]:
import time
import datetime
import operator
import urllib.request, json

import numpy as np
from oceanoobsbrasil.simcosta import Simcosta

In [52]:
s = Simcosta()

In [53]:
s.get()

id                  897
name                RJ3
lat            -22.9831
lon            -43.1745
data_type          buoy
institution    simcosta
url                   3
Name: 0, dtype: object
http://simcosta.furg.br/api/metereo_data?boiaID=3&type=json&time1=1631069870&time2=1631156270&params=Average_wind_direction_N,Last_sampling_interval_gust_speed,Average_Dew_Point,Average_Pressure,Solar_Radiation_Average_Reading,Average_Air_Temperature,Instantaneous_Humidity,Average_Humidity,Average_wind_speed
http://simcosta.furg.br/api/oceanic_data?boiaID=3&type=json&time1=1631069870&time2=1631156270&params=H10,HAvg,Hsig_Significant_Wave_Height_m,HM0,Mean_Wave_Direction_deg,Hmax_Maximum_Wave_Height_m,ZCN,Tp5,TAvg,T10,Tsig,Mean_Spread_deg,TP_Peak_Period_seconds,Average_Salinity,Average_Temperature_deg_C,Average_Temperature_C,Average_CDOM_QSDE,Average_Chlorophyll_Fluorescence,Average_Dissolved_Oxygen,Average_Nephelometric_Turbidity_Unit_NTU,Cell_Average_Direction_N,Cell_Average_Magnitude_mm_s
Empty D

    YEAR MONTH DAY HOUR MINUTE SECOND Avg_Air_Press Avg_Air_Tmp    Hmt  \
0   2021     9   8    3     21     00       1018.53        None  45.20   
1   2021     9   8    3     51     00       1018.39        None  44.30   
2   2021     9   8    4     51     00       1017.51        None  44.00   
3   2021     9   8    5     21     00       1017.30        None  44.60   
4   2021     9   8    5     51     00       1016.99        None  43.80   
5   2021     9   8    6     51     00       1015.98        None  43.70   
6   2021     9   8    7     21     00       1015.71        None  44.60   
7   2021     9   8    7     51     00       1015.55        None  43.60   
8   2021     9   8    8     21     00       1015.67        None  44.50   
9   2021     9   8    9     21     00       1016.01        None  44.50   
10  2021     9   8    9     51     00       1016.16        None  43.70   
11  2021     9   8   10     21     00       1016.07        None  45.40   
12  2021     9   8   10     51     00 

ok2
id                  903
name                BA1
lat            -12.9895
lon            -37.4582
data_type          buoy
institution    simcosta
url                  15
Name: 6, dtype: object
http://simcosta.furg.br/api/metereo_data?boiaID=15&type=json&time1=1631069870&time2=1631156270&params=Average_wind_direction_N,Last_sampling_interval_gust_speed,Average_Dew_Point,Average_Pressure,Solar_Radiation_Average_Reading,Average_Air_Temperature,Instantaneous_Humidity,Average_Humidity,Average_wind_speed
http://simcosta.furg.br/api/oceanic_data?boiaID=15&type=json&time1=1631069870&time2=1631156270&params=H10,HAvg,Hsig_Significant_Wave_Height_m,HM0,Mean_Wave_Direction_deg,Hmax_Maximum_Wave_Height_m,ZCN,Tp5,TAvg,T10,Tsig,Mean_Spread_deg,TP_Peak_Period_seconds,Average_Salinity,Average_Temperature_deg_C,Average_Temperature_C,Average_CDOM_QSDE,Average_Chlorophyll_Fluorescence,Average_Dissolved_Oxygen,Average_Nephelometric_Turbidity_Unit_NTU,Cell_Average_Direction_N,Cell_Average_Magnitude_mm_s
 

In [40]:
                columns = ['YEAR','MONTH','DAY','HOUR','MINUTE','SECOND','Hmt',
                    'Avg_Wnd_Dir','M_Decl','Avg_W_Tmp1','Avg_Sal','Avg_Spre_N',
                    'Avg_Wv_Dir','Avg_Cel1_Mag','Avg_Cel1_Dir','Avg_Cel1_Dir_N',
                    'Avg_Turb','Avg_Chl','Avg_DO','ZCN','HM0','TAvg','Tp5',
                    'T10','HAvg','Tsig','CDOM','H10','Avg_Sol_Rad']



In [43]:
s.result.drop(columns=columns, inplace=True)

In [44]:
s.result.columns = ['pres', 'atmp', 'rh', 'dewpt', 'wspd', 'wdir', 'gust', 'swvht', 'mxwvht', 'tp', 'sst', 'wvspread', 'wvdir', 'date_time']

In [45]:
s.result = s.result.replace(to_replace =['None', 'NULL', ' ', ''],
                                        value =np.nan)

In [46]:
s.result['station_id'] = 898

In [ ]:
s.